## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hilo,US,19.7297,-155.0900,88.29,86,100,13.80,moderate rain
1,1,Pitimbu,BR,-7.4706,-34.8086,73.99,81,4,10.36,clear sky
2,2,Lagoa,PT,39.0500,-27.9833,74.55,85,20,9.75,few clouds
3,3,Busselton,AU,-33.6500,115.3333,54.70,70,83,7.54,broken clouds
4,4,Nikolskoye,RU,59.7035,30.7861,67.91,83,48,5.48,scattered clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hilo,US,19.7297,-155.0900,88.29,86,100,13.80,moderate rain
7,7,Vaini,TO,-21.2000,-175.2000,75.36,100,75,13.80,broken clouds
8,8,Hambantota,LK,6.1241,81.1185,77.79,80,100,16.24,overcast clouds
10,10,Butaritari,KI,3.0707,172.7902,82.22,73,84,9.08,broken clouds
14,14,Faanui,PF,-16.4833,-151.7500,77.18,79,60,22.66,light rain
20,20,Hithadhoo,MV,-0.6000,73.0833,81.86,74,94,12.30,overcast clouds
21,21,Marsa Matruh,EG,31.3525,27.2453,77.20,78,20,4.61,few clouds
22,22,Nha Trang,VN,12.2500,109.1833,78.80,81,100,4.76,overcast clouds
24,24,Albany,US,42.6001,-73.9662,86.65,89,82,0.51,broken clouds
28,28,Kavieng,PG,-2.5744,150.7967,83.19,68,59,5.66,broken clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        254
City           254
Country        254
Lat            254
Lng            254
Max Temp       254
Humidity       254
Cloudiness     254
Wind Speed     254
Description    254
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis=0, how="any")

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Hilo,US,88.29,moderate rain,19.7297,-155.0900,
7,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,
8,Hambantota,LK,77.79,overcast clouds,6.1241,81.1185,
10,Butaritari,KI,82.22,broken clouds,3.0707,172.7902,
14,Faanui,PF,77.18,light rain,-16.4833,-151.7500,
20,Hithadhoo,MV,81.86,overcast clouds,-0.6000,73.0833,
21,Marsa Matruh,EG,77.20,few clouds,31.3525,27.2453,
22,Nha Trang,VN,78.80,overcast clouds,12.2500,109.1833,
24,Albany,US,86.65,broken clouds,42.6001,-73.9662,
28,Kavieng,PG,83.19,broken clouds,-2.5744,150.7967,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0, how="any")
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Hilo,US,88.29,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
7,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
8,Hambantota,LK,77.79,overcast clouds,6.1241,81.1185,Bungalow 63
10,Butaritari,KI,82.22,broken clouds,3.0707,172.7902,Isles Sunset Lodge
14,Faanui,PF,77.18,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))